# Harmony API Introduction

This notebook provides an overview of the capabilities offered through the Harmony API and SWOT L2 Reproject tool. While written for SWOT L2 data, it works with most any level 2 data for projecting to a normal grid. In this tutorial we will use MODIS L2 data to show the native file projected to equal-area-cylindracal projection using both Nearest Neighbor and Bi-linear interpolation.

Standing on the shoulders of previous authors: Amy Steiker, Patrick Quinn

## Before you start

Before you beginning this tutorial, make sure you have an account in the Earthdata Login, which is required to access data from the NASA Earthdata system. Please visit https://urs.earthdata.nasa.gov to register for an Earthdata Login account. It is free to create and only takes a moment to set up.

You will also need a netrc file containing your NASA Earthdata Login credentials in order to execute this notebook. A netrc file can be created manually within text editor and saved to your home directory. For additional information see: [Authentication for NASA Earthdata](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/04_NASA_Earthdata_Authentication.html#authentication-via-netrc-file).

## Import packages

In [2]:
from urllib import request, parse
from http.cookiejar import CookieJar
import getpass
import netrc
import os
import requests
import json
import pprint
from osgeo import gdal
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import rasterio
from rasterio.plot import show
import numpy as np
import os
import time
from netCDF4 import Dataset
%matplotlib inline

## Identify a data collection of interest

A CMR collection ID is needed to request services through Harmony. The collection ID can be determined using the [CMR API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html). We will query the corresponding ID of a known collection short name, `MODIS_A-JPL-L2P-v2019.0`.

In [5]:
params = {
    'short_name': 'MODIS_A-JPL-L2P-v2019.0',
    'provider_id': 'POCLOUD'
} # parameter dictionary with known CMR short_name

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
cmr_response = requests.get(cmr_collections_url, params=params)
cmr_results = json.loads(cmr_response.content) # Get json response from CMR collection metadata

collectionlist = [el['id'] for el in cmr_results['feed']['entry']]
harmony_collection_id = collectionlist[0]
print(harmony_collection_id)

C1940473819-POCLOUD


We can also view the `MODIS_A-JPL-L2P-v2019.0` collection metadata to glean more information about the collection:

In [6]:
pprint.pprint(cmr_results)

{'feed': {'entry': [{'archive_center': 'NASA/JPL/PODAAC',
                     'associations': {'services': ['S1962070864-POCLOUD',
                                                   'S2004184019-POCLOUD',
                                                   'S2153799015-POCLOUD',
                                                   'S2227193226-POCLOUD'],
                                      'tools': ['TL2108419875-POCLOUD',
                                                'TL2092786348-POCLOUD'],
                                      'variables': ['V1997812737-POCLOUD',
                                                    'V1997812697-POCLOUD',
                                                    'V2112014688-POCLOUD',
                                                    'V1997812756-POCLOUD',
                                                    'V1997812688-POCLOUD',
                                                    'V1997812670-POCLOUD',
                                                  

Next we get a granule ID from this collection, G2525170359-POCLOUD.

In [11]:
cmr_url = "https://cmr.earthdata.nasa.gov/search/granules.umm_json?collection_concept_id="+harmony_collection_id+"&sort_key=-start_date"

response = requests.get(cmr_url)

gid=response.json()['items'][0]['meta']['concept-id']
print(gid)

G2525170359-POCLOUD


## Access reprojected data

The Harmony API accepts reprojection requests with a given coordinate reference system using the `outputCrs` keyword. According to the Harmony API documentation, this keyword "recognizes CRS types that can be inferred by gdal, including EPSG codes, Proj4 strings, and OGC URLs (http://www.opengis.net/def/crs/...) ". 

## The practice datasets below used for this tutorial are no longer supported, for details about the Harmony API see [this tutorial](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/07_Harmony_Subsetting.html#data-subsetting-and-transformation-services-in-the-cloud) from the 2021 Cloud Hackathon or [this tutorial](https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb) introducing the Harmony-py library.
Two examples below demonstrate inputting an EPSG code and Proj4 string using the global test granule from previous examples. First, let's view the projection information of the granule in the native projection, using the variable subset example:

## Access Level 2 swath regridded data

Moving outside of the `harmony/gdal` service, we will now request regridding from the `sds/swot-reproject` service using the `C1940473819-POCLOUD`. 


The Harmony API accepts several query parameters related to regridding and interpolation in addition to the reprojection parameters above: 

`interpolation=<String>` - Both `near` and `bilinear` are valid options

`scaleSize=x,y` - 2 comma separated numbers as floats

`scaleExtent=xmin,ymin,xmax,ymax` - 4 comma separated numbers as floats

`width=<Float>`  

`height=<Float>` 

An error is returned if both `scaleSize` and `width`/`height` parameters are both provided (only one or the other can be used).

Request reprojection to [Europe Lambert Conformal Conic](https://epsg.io/102014) with a new scale extent and nearest neighbor interpolation:

In [13]:
harmony_root = 'https://harmony.earthdata.nasa.gov'

# URL encode string using urllib parse package
proj_string = '+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs' # proj4 of WGS 84 / NSIDC EASE-Grid 2.0 Global projection
#l2proj_string = '+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'
l2proj_encode = parse.quote(proj_string)

regridConfig = {
    'l2collection_id': 'C1940473819-POCLOUD',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1234734747-POCLOUD',
    'outputCrs': l2proj_encode,
    'interpolation': 'near',
    'width': 1000,
    'height': 1000
}

regrid_url = harmony_root+'/{l2collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?&granuleid={granuleid}&outputCrs={outputCrs}&interpolation={interpolation}&height={height}&width={width}'.format(**regridConfig)
print('Request URL', regrid_url)
regrid_response = request.urlopen(regrid_url)
regrid_results = regrid_response.read()

Request URL https://harmony.earthdata.nasa.gov/C1940473819-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?&granuleid=G1234734747-POCLOUD&outputCrs=%2Bproj%3Dcea%20%2Blon_0%3D0%20%2Blat_ts%3D30%20%2Bx_0%3D0%20%2By_0%3D0%20%2Bellps%3DWGS84%20%2Btowgs84%3D0%2C0%2C0%2C0%2C0%2C0%2C0%20%2Bunits%3Dm%20%2Bno_defs&interpolation=near&height=1000&width=1000


HTTPError: HTTP Error 401: Unauthorized

This reprojected and regridded output is downloaded to the Harmony outputs directory and we can inspect a variable to check for projection and grid dimension:

In [ ]:
regrid_file_name = 'regrid-near.nc'
regrid_filepath = str(regrid_file_name)
file_ = open(regrid_filepath, 'wb')
file_.write(regrid_results)
file_.close()

In [ ]:
harmony_root = 'https://harmony.earthdata.nasa.gov'

# URL encode string using urllib parse package
proj_string = '+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs' # proj4 of WGS 84 / NSIDC EASE-Grid 2.0 Global projection
#l2proj_string = '+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'
l2proj_encode = parse.quote(proj_string)

harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

regridConfig = {
    'l2collection_id': 'C1234724470-POCLOUD',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1234734747-POCLOUD',
    'outputCrs': l2proj_encode,
    'interpolation': 'bilinear',
    'width': 1000,
    'height': 1000
}

regrid_bi_url = harmony_root+'/{l2collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?&granuleid={granuleid}&outputCrs={outputCrs}&interpolation={interpolation}&height={height}&width={width}'.format(**regridConfig)
print('Request URL', regrid_bi_url)
regrid_bi_response = request.urlopen(regrid_bi_url)
regrid_bi_results = regrid_bi_response.read()

In [ ]:
regrid_bi_file_name = 'regrid-bi.nc'
regrid_bi_filepath = str(regrid_bi_file_name)
file_ = open(regrid_bi_filepath, 'wb')
file_.write(regrid_bi_results)
file_.close()

Print the x and y dimensions to confirm that the output matches the requested scale extent in meters:

In [ ]:
import xarray as xr
reproject_ds = xr.open_dataset(regrid_filepath, drop_variables='time')
print(reproject_ds)

In [ ]:
import xarray as xr
reproject_bi_ds = xr.open_dataset(regrid_bi_filepath, drop_variables='time')
print(reproject_bi_ds)

In [ ]:
original_ds = xr.open_dataset('20200131234501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc')
print(original_ds)

In [ ]:
g = reproject_ds.sea_surface_temperature.plot(robust=True)
g.axes.set_title("Nearest Neighbor Interpolation")

In [ ]:
g= reproject_bi_ds.sea_surface_temperature.plot(robust=True)
g.axes.set_title("Bilinear Interpolation")

In [ ]:
g = original_ds.sea_surface_temperature.plot(robust=True)
g.axes.set_title("Native File")


In [ ]:
g= original_ds.sea_surface_temperature.plot(x="lon", y="lat", robust=True)
g.axes.set_title("Native, projected to Lat/Lon")